In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import random

In [2]:
train_df = pd.read_csv("../input/choose-tutors/train.csv")
test_df = pd.read_csv("../input/choose-tutors/test.csv")

train_df.head()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
0,0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0
1,1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1
2,2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0
3,3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,1
4,4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0


In [3]:
class Node:
    """Класс узла"""
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

class Leaf:
    """Класс терминального узла (листа)"""
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction

class RandomForestClassifier():
    """Класс алгоритма RandomForestClassifier"""

    @property
    def forest_(self):
        return self.forest
    
    @property
    def n_trees_(self):
        return self.n_trees

    def __init__(self, n_trees = 1, max_depth=5, min_leaf=1, random_state=42):
        self.n_trees = n_trees
        self.random_state = random_state
        self.max_depth = max_depth
        self.min_leaf = min_leaf
        self.forest = []

    def bootstrap(self, X, y):
        n_samples = X.shape[0]
        np.random.seed(self.random_state)
        
        bootstrap = []
        for i in range(self.n_trees):
            b_data = np.zeros(X.shape)
            b_labels = np.zeros(y.shape)
            
            for j in range(n_samples):
                sample_index = np.random.randint(0, n_samples-1)
                b_data[j] = X[sample_index]
                b_labels[j] = y[sample_index]
            bootstrap.append((b_data, b_labels))
        
        return bootstrap
    
    def subsample(self, len_sample):
        # будем сохранять не сами признаки, а их индексы
        sample_indexes = [i for i in range(len_sample)]
        
        len_subsample = int(np.sqrt(len_sample))
        
        subsample = []
        np.random.shuffle(sample_indexes)
        for _ in range(len_subsample):
            subsample.append(sample_indexes.pop())
        
        return subsample
    
    def calc_criterion(self, y):
        """Расчет критерия информативности"""

        #  подсчет количества объектов разных классов
        classes = {}
        for label in y:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1

        # Расчет критерия Джини
        impurity = 1
        for label in classes:
            p = classes[label] / len(y)
            impurity -= p ** 2

        return impurity
    
    def quality(self, left_labels, right_labels, current_criterion):
        """Расчет качества"""

        # доля выбоки, ушедшая в левое поддерево
        p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
        
        return current_criterion - p * self.calc_criterion(left_labels) - (1 - p) * self.calc_criterion(right_labels)

    @staticmethod
    def split(X, y, index, t):
        """Разбиение датасета в узле"""

        left = np.where(X[:, index] <= t)
        right = np.where(X[:, index] > t)
            
        true_data = X[left]
        false_data = X[right]
        true_labels = y[left]
        false_labels = y[right]
            
        return true_data, false_data, true_labels, false_labels

    def find_best_split(self, X, y):
        """Нахождение наилучшего разбиения"""

        current_criterion = self.calc_criterion(y)

        best_quality = 0
        best_t = None
        best_index = None
        
        n_features = X.shape[1]
        
        # выбор индекса из подвыборки длиной sqrt(n_features)
        subsample = self.subsample(n_features)
        
        for index in subsample:
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique([row[index] for row in X])
            
            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(X, y, index, t)
                #  пропускаем разбиения, в которых в узле объектов менее заданных в min_leaf
                if len(true_data) < self.min_leaf or len(false_data) < self.min_leaf:
                    continue
                
                current_quality = self.quality(true_labels, false_labels, current_criterion)
                
                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index

    def tree(self, X, y):

        def build_tree(X, y, **kwargs):
            """Построение дерева с помощью рекурсивной функции"""
            
            # ограничение по глубине дерева
            kwargs['depth'] += 1
            if kwargs['depth'] > self.max_depth:
                return Leaf(X, y)

            quality, t, index = self.find_best_split(X, y)

            #  Базовый случай - прекращаем рекурсию, когда нет прироста качества
            if quality == 0:
                return Leaf(X, y)

            true_data, false_data, true_labels, false_labels = self.split(X, y, index, t)

            # Рекурсивно строим два поддерева
            true_branch = build_tree(true_data, true_labels, depth=kwargs['depth'])
            false_branch = build_tree(false_data, false_labels, depth=kwargs['depth'])
            
            # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
            return Node(index, t, true_branch, false_branch)
    
        return build_tree(X, y, depth=0)
    
    def predict(self, X, proba=False):
        """Предсказание голосованием деревьев """
        
        def tree_predict(X, tree):
            """Функция формирования предсказания по выборке на одном дереве"""
            
            return [classify_object(obj, tree) for obj in X]
        
        def classify_object(obj, node):
            """Функция классификации отдельного объекта"""

            #  Останавливаем рекурсию, если достигли листа
            if isinstance(node, Leaf):
                return node.prediction
            
            if obj[node.index] <= node.t:
                return classify_object(obj, node.true_branch)
            else:
                return classify_object(obj, node.false_branch)
        
        # добавим предсказания всех деревьев в список
        predictions = []
        for tree in self.forest:
            prediction = tree_predict(X, tree)
            predictions.append(prediction)
        
        # сформируем список с предсказаниями для каждого объекта
        predictions_per_object = list(zip(*predictions))
        
        answers = []
        if proba is True:
            for obj in predictions_per_object:
                class_0 = 0
                class_1 = 0
                for itm in obj:
                    if itm == 0:
                        class_0 += 1
                    else:
                        class_1 += 1
                probability = class_1 / (class_0 + class_1)
                answers.append(probability)
        else:
            for obj in predictions_per_object:
                predicted_class = max(set(obj), key=obj.count)
                answers.append(predicted_class)
        
        return answers

    def predict_proba(self, X):
        """Получение вероятности предсказания"""
        
        return self.predict(X, proba=True)
    
    def fit(self, X, y):
        bootstrap = self.bootstrap(X, y)
        
        for b_data, b_labels in bootstrap:
            self.forest.append(self.tree(b_data, b_labels))

In [4]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


def std_func(x):    
    return (x - x.mean()) / x.std()

In [6]:
def results(model, X_train, X_test, y_train, y_test):

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    print('Train')
    print(f'Accuracy: {accuracy_metric(y_train, y_train_pred):.3f}')
    
    print('\nTest')
    print(f'Accuracy: {accuracy_metric(y_test, y_test_pred):.3f}')

In [7]:
np_train = train_df.to_numpy()

X = std_func(np_train[:,1:12])
y = np_train[:,-1].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    shuffle=True,
                                                    test_size=0.25,
                                                    random_state=0,
                                                    stratify=y)

In [9]:
clf = RandomForestClassifier(
    n_trees=50,
    max_depth=7,
    min_leaf=1,
    random_state=0,
)
clf.fit(X_train, y_train)

In [10]:
results(clf, X_train, X_test, y_train, y_test)

Train
Accuracy: 89.720

Test
Accuracy: 89.480


In [12]:
np_test = test_df.to_numpy()
X_test = std_func(np_test[:,1:12])
predictions = clf.predict_proba(X_test)

In [13]:
submit = pd.read_csv('../input/choose-tutors/submission_example.csv')
submit.head()

,Id,choose
0,10000,0.5
1,10001,0.5
2,10002,0.5
3,10003,0.5
4,10004,0.5


In [14]:
submit['choose'] = predictions
submit.head()

,Id,choose
0,10000,0.24
1,10001,0.46
2,10002,0.24
3,10003,0.24
4,10004,0.40


In [15]:
submit.to_csv('rf_submit.csv', index=False)